task
data
modeling
error measurement
training and optimization
performance measurement
validating and testing
corrective measures

In [1]:
import tensorflow as tf
import pandas as pd #reading and processing data
import seaborn as sns #plotting and visualizing data
from tensorflow.keras.layers import Normalization, Dense,InputLayer, Input
from tensorflow.keras.losses import MeanSquaredError,MeanAbsoluteError,Huber
import pydot
import graphviz

**Data Preperation**

In [2]:
data = pd.read_csv("D:/TENSOR FLOW/carPricePrediction/train.csv", sep=",")
data.head()
print(data.shape)  

(1000, 12)


In [3]:
datap = pd.read_csv("D:/TENSOR FLOW/carPricePrediction/train_semi.csv", sep=";")
print(datap.head())
print(datap.shape)

   v.id  on road old  on road now  years     km  rating  condition  economy  \
0     1       535651       798186      3  78945       1          2       14   

   top speed  hp  torque  current price  
0        177  73     123         351318  
(1, 12)


In [4]:
sns.pairplot(data[['years', 'km', 'rating', 'condition', 'economy', 'top speed', 'hp', 'torque', 'current price']], diag_kind='kde')

In [ ]:
tensor_data = tf.constant(data.values)
tensor_data = tf.cast(tensor_data, tf.float32)
print(tensor_data)


tf.Tensor(
[[1.000000e+00 5.356510e+05 7.981860e+05 ... 7.300000e+01 1.230000e+02
  3.513180e+05]
 [2.000000e+00 5.919110e+05 8.610560e+05 ... 7.400000e+01 9.500000e+01
  2.850015e+05]
 [3.000000e+00 6.869900e+05 7.707620e+05 ... 5.300000e+01 9.700000e+01
  2.153860e+05]
 ...
 [9.980000e+02 6.463440e+05 8.427330e+05 ... 1.130000e+02 8.900000e+01
  4.058710e+05]
 [9.990000e+02 5.355590e+05 7.324390e+05 ... 1.120000e+02 1.280000e+02
  7.439800e+04]
 [1.000000e+03 5.901050e+05 7.797430e+05 ... 9.900000e+01 9.600000e+01
  4.149385e+05]], shape=(1000, 12), dtype=float32)


so to avoid any bias by the way in which data is collected, we shuffle the data

In [ ]:
tensor_data = tf.random.shuffle(tensor_data)
print(tensor_data[:5])

tf.Tensor(
[[9.660000e+02 6.742740e+05 8.778200e+05 6.000000e+00 1.193560e+05
  2.000000e+00 7.000000e+00 1.300000e+01 1.940000e+02 5.900000e+01
  1.070000e+02 3.034190e+05]
 [3.500000e+01 6.304890e+05 7.171820e+05 5.000000e+00 9.622600e+04
  1.000000e+00 9.000000e+00 8.000000e+00 1.910000e+02 1.010000e+02
  9.400000e+01 3.171190e+05]
 [3.620000e+02 6.986120e+05 8.680920e+05 4.000000e+00 1.398590e+05
  3.000000e+00 6.000000e+00 1.400000e+01 1.880000e+02 5.400000e+01
  7.300000e+01 2.276690e+05]
 [5.700000e+02 6.014620e+05 7.001770e+05 3.000000e+00 6.977900e+04
  4.000000e+00 6.000000e+00 1.500000e+01 1.760000e+02 5.600000e+01
  1.140000e+02 3.762675e+05]
 [7.370000e+02 5.309510e+05 7.410860e+05 4.000000e+00 1.163180e+05
  3.000000e+00 8.000000e+00 1.500000e+01 1.710000e+02 1.060000e+02
  1.180000e+02 1.870500e+05]], shape=(5, 12), dtype=float32)


In [ ]:
x = tensor_data[:,3:-1]
print(x.shape)
print(x[:5])


y = tensor_data[:,-1]
print(y[:5].shape)
y = tf.expand_dims(y, axis = -1)
print(y[:5])

(1000, 8)
tf.Tensor(
[[6.00000e+00 1.19356e+05 2.00000e+00 7.00000e+00 1.30000e+01 1.94000e+02
  5.90000e+01 1.07000e+02]
 [5.00000e+00 9.62260e+04 1.00000e+00 9.00000e+00 8.00000e+00 1.91000e+02
  1.01000e+02 9.40000e+01]
 [4.00000e+00 1.39859e+05 3.00000e+00 6.00000e+00 1.40000e+01 1.88000e+02
  5.40000e+01 7.30000e+01]
 [3.00000e+00 6.97790e+04 4.00000e+00 6.00000e+00 1.50000e+01 1.76000e+02
  5.60000e+01 1.14000e+02]
 [4.00000e+00 1.16318e+05 3.00000e+00 8.00000e+00 1.50000e+01 1.71000e+02
  1.06000e+02 1.18000e+02]], shape=(5, 8), dtype=float32)
(5,)
tf.Tensor(
[[303419. ]
 [317119. ]
 [227669. ]
 [376267.5]
 [187050. ]], shape=(5, 1), dtype=float32)


**Normalizing the data**

In [ ]:
normalizer = Normalization()
normalizer.adapt(x)
normalizer(x)[:5]


<tf.Tensor: shape=(5, 8), dtype=float32, numpy=
array([[ 0.8374948 ,  0.6549165 , -0.70466214,  0.4987537 ,  0.6167485 ,
         1.4060569 , -1.2457402 ,  0.16994357],
       [ 0.25549707, -0.13894996, -1.417883  ,  1.2072107 , -1.6259733 ,
         1.250445  ,  0.80237275, -0.4476869 ],
       [-0.32650065,  1.3586192 ,  0.00855871,  0.14452522,  1.065293  ,
         1.0948331 , -1.4895632 , -1.4453977 ],
       [-0.9084984 , -1.0466623 ,  0.7217796 ,  0.14452522,  1.5138373 ,
         0.47238556, -1.3920339 ,  0.5025138 ],
       [-0.32650065,  0.5506464 ,  0.00855871,  0.85298216,  1.5138373 ,
         0.2130324 ,  1.0461957 ,  0.692554  ]], dtype=float32)>

**Linear Regression Model** 

In [ ]:
model = tf.keras.Sequential([
    #Input(shape=(x.shape[1],)), # Input function 
    InputLayer(input_shape = (8,)),
    normalizer,
    Dense(1)
])
model.summary()

c:\Users\chaga\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_104"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_86                │ (None, 8)              │            17 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26 (108.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 17 (72.00 B)

In [ ]:
tf.keras.utils.plot_model(model, to_file="model.png" , show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
model.compile(loss= MeanSquaredError())
model.compile(loss = Huber())
model.compile(loss = MeanAbsoluteError())

SGD
W = Wp - (LR) rho